In [1]:
import os
import datetime

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers

import dataset

In [2]:
MAXLEN = 60
BATCH_SIZE = 32
files = ['texts/' + f for f in os.listdir('texts/') if not f.startswith('.')]

data = dataset.load_file(BATCH_SIZE, 0.05, maxlen=MAXLEN, filenames=files)

In [3]:
EMBED_DIM = 512
UNITS = 400

common_input = tf.keras.Input(batch_shape=(None, data.input_texts.shape[1]), batch_size=BATCH_SIZE)
common = layers.Embedding(len(data.letters_table), EMBED_DIM, mask_zero=True)(common_input)
common = layers.Bidirectional(layers.GRU(UNITS, return_sequences=True, dropout=0.3), merge_mode='sum')(common)

common = layers.add([common, layers.Bidirectional(layers.GRU(UNITS, return_sequences=True, dropout=0.1), merge_mode='sum')(common)])

niqqud = layers.Softmax(name='N')(layers.Dense(data.niqqud_texts.shape[1])(common))
dagesh = layers.Softmax(name='D')(layers.Dense(data.dagesh_texts.shape[1])(common))

model = tf.keras.Model(inputs=[common_input], outputs=[niqqud, dagesh])

tf.keras.utils.plot_model(model, to_file='model.png')
model.summary()

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 60)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 60, 512)      37376       input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 60, 400)      2193600     embedding[0][0]                  
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 60, 400)      1924800     bidirectional[0][0]              
________

In [4]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              metrics=['accuracy'])

def fit(learning_rates):
    return model.fit(data.input_texts, [data.niqqud_texts, data.dagesh_texts],
          batch_size=BATCH_SIZE,
          epochs=len(learning_rates),
          validation_data=(data.input_validation, [data.niqqud_validation,  data.dagesh_validation]),
          callbacks=[
              tf.keras.callbacks.LearningRateScheduler(lambda epoch, lr: learning_rates[epoch], verbose=0),
              # tf.keras.callbacks.ModelCheckpoint(filepath='checkpoints/ckpt_{epoch}', save_weights_only=True),
          ]
    )

In [ ]:
history = fit([2e-3, 7e-4]) #, 1e-4, 3e-6])

Train on 111720 samples, validate on 5880 samples
Epoch 1/2
111720/111720 [==============================] - 178s 2ms/sample - loss: 0.2937 - N_loss: 0.2323 - D_loss: 0.0614 - N_accuracy: 0.9213 - D_accuracy: 0.9774 - val_loss: 0.1815 - val_N_loss: 0.1402 - val_D_loss: 0.0413 - val_N_accuracy: 0.9532 - val_D_accuracy: 0.9850
Epoch 2/2
 65760/111720 [================>.............] - ETA: 1:08 - loss: 0.1404 - N_loss: 0.1082 - D_loss: 0.0322 - N_accuracy: 0.9636 - D_accuracy: 0.9880- ETA: 1:09 - loss: 0.1405 - N_loss: 0.1083 - D_loss: 0.0322 - N_accuracy: 0.9635 - D_accurac

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2)

for n, v in enumerate(['accuracy', 'loss'], 0):
    for n1, t in enumerate(['D', 'N'], 0):
        p = ax[n][n1]
        p.plot(history.history[t + '_' + v][0:])
        p.plot(history.history['val_' + t + '_' +  v][0:])
        p.legend([t + '_Train', t + '_Test'], loc='center right')

plt.tight_layout()

In [ ]:
def print_predictions(k):
    s = slice(k*BATCH_SIZE, (k+1)*BATCH_SIZE)
    batch = data.input_validation[s]
    [actual_niqqud, actual_dagesh] = dataset.from_categorical(model.predict(batch))
    [expected_niqqud, expected_dagesh] = [data.niqqud_validation[s], data.dagesh_validation[s]]
    actual = data.merge(batch, ns=actual_niqqud, ds=actual_dagesh)
    expected = data.merge(batch, ns=expected_niqqud, ds=expected_dagesh)
    for i, (a, e) in enumerate(zip(actual, expected)):
        print('מצוי: ', a)
        print('רצוי: ', e)
        print()

print_predictions(8)